# DeepSpeed

[MicroSoft DeepSpeed](https://github.com/microsoft/DeepSpeed)

[참고한 블로그: DeepSpeed로 큰 모델 튜닝하기](https://velog.io/@seoyeon96/DeepSpeed%EB%A1%9C-%ED%81%B0-%EB%AA%A8%EB%8D%B8-%ED%8A%9C%EB%8B%9D%ED%95%98%EA%B8%B0)

## DeepSpeed 환경세팅

In [9]:
# Runtime: T4 GPU
!nvidia-smi

Wed Apr 10 14:36:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install deepspeed
# !git clone https://github.com/microsoft/DeepSpeed 으로 수동으로 설치하는 방법도 있다.

In [21]:
# 현재 환경과 호환되는 옵션
!ds_report

[2024-04-10 14:50:57,402] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables

In [30]:
# CUDA version check
!nvcc --version
# 12.2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [35]:
# TORCH_CUDA_ARCH_LIST 버전 체크
CUDA_VISIBLE_DEVICES=0
!python -c "import torch; print(torch.cuda.get_device_capability())"

(7, 5)


**DeepSpeed Ops 사전 설치**

DS_BUILD_ADAM: CPUAdam을 구축

In [39]:
%%bash
TORCH_CUDA_ARCH_LIST="7.5" DS_BUILD_CPU_ADAM=1 DS_BUILD_UTILS=1 pip install . \
--global-option="build_ext" --global-option="-j8" --no-cache -v \
--disable-pip-version-check 2>&1 | tee build.log

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/DeepSpeed
  Preparing metadata (setup.py): started
  Running command python setup.py egg_info
  [2024-04-10 15:04:22,142] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
  [2024-04-10 15:04:22,358] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
   [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
   [WARNING]  async_io: please install the libaio-dev package with apt
   [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
   [WARNING]  P

In [43]:
# 병렬처리를 하기 위해 mpi4py 패키지 설치
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746503 sha256=28563544c21c7e1fe5bb989d62accc35d8c53daa7f20c0ea1747bcfc4817c8e8
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


**DeepSpeed에서 사용할 모델 설정 관련 참고 자료**

[Fine-Tuning Llama-2 LLM on Google Colab: A Step-by-Step Guide.](https://medium.com/@csakash03/fine-tuning-llama-2-llm-on-google-colab-a-step-by-step-guide-cf7bb367e790)

In [ ]:
# model requirment download
!pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate bitsandbytes

## Getting Started

In [59]:
# DeepSpeed Engine 초기화
import deepspeed
deepspeed.init_distributed()

In [87]:
# DeepSpeed Configuration
cmd_args = {
  "train_batch_size": 8,
  "gradient_accumulation_steps": 1,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 0.00015
    }
  },
  "fp16": {
    "enabled": True
  },
  "zero_optimization": True
}

In [69]:
# BART model import
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, PretrainedConfig, AutoModel, AutoModelForPreTraining, AutoTokenizer, AutoModelForSeq2SeqLM

model_name_or_path = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name_or_path)
model = BartForConditionalGeneration.from_pretrained(model_name_or_path)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [91]:
print(cmd_args)

{'train_batch_size': 8, 'gradient_accumulation_steps': 1, 'optimizer': {'type': 'Adam', 'params': {'lr': 0.00015}}, 'fp16': {'enabled': True}, 'zero_optimization': True}


In [93]:
# DeepSpeed 엔진을 초기화합니다.
model_engine, optimizer, _, _ = deepspeed.initialize(args=cmd_args,
                                                     model=model,
                                                     model_parameters=model.parameters(),
                                                     config_params=cmd_args)
# AssertionError: DeepSpeed requires --deepspeed_config to specify configuration file
# DeepSpeed가 --deepspeed_config 옵션을 찾기 위해 구성 파일을 요구함 -> config_params=cmd_args 으로 직접 매개변수를 구

[2024-04-10 16:09:56,129] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.1, git-hash=[none], git-branch=[none]
[2024-04-10 16:09:56,132] [WARNING] [config.py:57:read_zero_config_deprecated] DeepSpeedConfig: this format of ZeRO optimization setup is deprecated. Please use the following format: 
ZeRO optimization should be enabled as:
"session_params": {
  "zero_optimization": {
    "stage": [0|1|2],
    "stage3_max_live_parameters" : 1000000000,
    "stage3_max_reuse_distance" : 1000000000,
    "allgather_partitions": [true|false],
    "use_multi_rank_bucket_allreduce": [true|false],
    "allgather_bucket_size": 500000000,
    "reduce_scatter": [true|false],
    "contiguous_gradients" : [true|false]
    "overlap_comm": [true|false],
    "reduce_bucket_size": 500000000,
    "load_from_fp32_weights": [true|false],
    "cpu_offload": [true|false] (deprecated),
    "cpu_offload_params" : [true|false] (deprecated),
    "cpu_offload_use_pin_memory": [true|false] (deprec

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/fused_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Time to load fused_adam op: 74.58320951461792 seconds
[2024-04-10 16:11:12,497] [INFO] [logging.py:96:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2024-04-10 16:11:12,498] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2024-04-10 16:11:12,673] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam


Loading extension module fused_adam...


[2024-04-10 16:11:12,677] [INFO] [utils.py:56:is_zero_supported_optimizer] Checking ZeRO support for optimizer=FusedAdam type=<class 'deepspeed.ops.adam.fused_adam.FusedAdam'>
[2024-04-10 16:11:12,686] [INFO] [logging.py:96:log_dist] [Rank 0] Creating torch.float16 ZeRO stage 1 optimizer
[2024-04-10 16:11:12,690] [INFO] [stage_1_and_2.py:148:__init__] Reduce bucket size 500,000,000
[2024-04-10 16:11:12,700] [INFO] [stage_1_and_2.py:149:__init__] Allgather bucket size 500000000
[2024-04-10 16:11:12,703] [INFO] [stage_1_and_2.py:150:__init__] CPU Offload: False
[2024-04-10 16:11:12,711] [INFO] [stage_1_and_2.py:151:__init__] Round robin gradient partitioning: False
[2024-04-10 16:11:14,984] [INFO] [utils.py:773:see_memory_usage] Before initializing optimizer states
[2024-04-10 16:11:14,990] [INFO] [utils.py:774:see_memory_usage] MA 2.27 GB         Max_MA 3.03 GB         CA 3.03 GB         Max_CA 3 GB 
[2024-04-10 16:11:14,994] [INFO] [utils.py:781:see_memory_usage] CPU Virtual Memory:  u

## Training

In [103]:
from datasets import load_dataset

# IMDb 데이터셋을 로드합니다.
imdb_dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [104]:
train_dataset = imdb_dataset['train']
test_dataset = imdb_dataset['test']

In [107]:
train_dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_engine.to(device)

DeepSpeedEngine(
  (module): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50264, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50264, 1024, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096,

In [121]:
model_engine.train()

In [ ]:
for step, batch in enumerate(train_dataset):
    text = batch["text"]

    # 텍스트 데이터를 토큰화하고 인코딩하여 모델에 입력할 형식으로 변환합니다.
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    #forward() method
    outputs = model_engine(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

    #runs backpropagation
    # 역전파를 수행하고 가중치를 업데이트합니다.
    model_engine.backward(loss)

    # DeepSpeed에서 가중치 업데이트를 수행합니다.
    model_engine.step()

    # 일정 간격으로 손실을 출력합니다.
    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss.item()}")


model_engine.eval()